In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin) :    
    def __init__(self, columns):
        self.columns=columns
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.columns].values


In [181]:
class AgeImputer(BaseEstimator, TransformerMixin) :
    titleAgeDictionary={'Master':0, 'Miss':0, 'Mr.':0, 'Mrs':0}
        
    def fit(self, X, y=None):
        for key in self.titleAgeDictionary.keys():
            self.titleAgeDictionary[key] = X[ np.char.find(X[:,0], key) ].astype(np.float).median()
            
        self.titleAgeDictionary['any'] = X[:,1].astype(np.float).median()
        return self
    
    def findAge(self, r) :
        if (r['Age'] > 0):
            return r
        for key in self.titleAgeDictionary.keys():
            if (r['Name'].find(key) != -1):
                r["Age"]=self.titleAgeDictionary[key]
                return r;
        r['Age'] = self.titleAgeDictionary['any']
        return r
        
    def transform(self, X, y=None):        
        return X.apply(self.findAge, axis=1)
        

In [50]:
#test
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

%matplotlib inline

# read the dataframe
df = pd.read_csv('train.csv')
f = lambda x : 0 if ( x != x) else 1;
df['Cabin'] = df['Cabin'].apply(f)

dfSelector = DataFrameSelector(['Name', 'Age'])
r = dfSelector.fit_transform(df)

x = r[np.where('Miss' in r[1])]





TypeError: argument of type 'float' is not iterable

In [198]:
def f(x):
    print(x)
    return False

x = np.array([['Master Bibi', np.NAN], 
             ['Mr. Hoto', 'Master'],
             ['Johnny', 3]])
# print(x)
# y = x[np.char.find(x[:,0], 'alp') != -1][:,1]
# y.astype(np.float).mean()
# # y = [1,2,3]
# y.tolist()

titleAgeDictionary={'Master':1, 'Miss':2, 'Mr.':3, 'Mrs':4, 'any':10}

# for key in titleAgeDictionary.keys():
x[np.char.find(x[:,0], key)!=-1 & x[:,1] ]